The NN structure is below.
x(42dim) -> x_1(128dim) ->x_2(1dim)

> Add blockquote



In [ ]:
using DelimitedFiles

In [ ]:
lr=0.000001

function activation(x)
    return max.(0, x)
end

function activation_derivative(x)
    return x .> 0
end

function forward(x,W1,b_1,W2,b_2)
    z_1=W1*x+b_1
    x_1=activation(z_1)
    z_2=W2*x_1+b_2
    x_2=z_2
    return z_1,x_1,x_2
end

function backward(loss,x,z_1,x_1,x_2,W1,b_1,W2,b_2,lr)
    W2-=lr.*(loss*x_1')
    b_2-=lr.*loss
    delta_1=(W2'*loss).*activation_derivative(z_1)

    W1-=lr.*(delta_1*x')
    b_1.-=lr.*delta_1
    return W1,b_1,W2,b_2
end

backward (generic function with 1 method)

In [ ]:
raw_train=readdlm("/content/train.csv",',')[:,2:end]
raw_test=readdlm("/content/test.csv",',')[:,2:end]
header=raw_train[1,:]
target_index=findfirst(x->x=="diagnosed_diabetes", header)
feature_indices=filter(i->i != target_index,1:length(header))
println(feature_indices)
train=raw_train[2:end,feature_indices]
valid=raw_train[2:end,target_index]
println(train[1,1:end])
println(valid[1,1:end])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Any[31, 1, 45, 7.7, 6.8, 6.1, 33.4, 0.93, 112, 70, 62, 199, 58, 114, 102, "Female", "Hispanic", "Highschool", "Lower-Middle", "Current", "Employed", 0, 0, 0]
Any[1.0]


In [ ]:
data=[train;raw_test[2:end,:]]

1000000×24 Matrix{Any}:
 31  1   45  7.7  6.8   6.1  33.4  0.93  …  "Current"  "Employed"  0  0  0
 50  2   73  5.7  6.5   5.8  23.8  0.83     "Never"    "Employed"  0  0  0
 32  3  158  8.5  7.4   9.1  24.1  0.83     "Never"    "Retired"   0  0  0
 54  3   77  4.6  7.0   9.2  26.6  0.83     "Current"  "Employed"  0  1  0
 54  1   55  5.7  6.2   5.1  28.8  0.9      "Never"    "Retired"   0  1  0
 42  1  100  4.4  6.4   5.3  25.5  0.84  …  "Never"    "Retired"   0  0  0
 41  2  148  3.4  5.6   3.7  27.9  0.89     "Current"  "Employed"  0  0  0
 51  3  102  4.0  7.3   5.5  27.1  0.83     "Never"    "Employed"  1  0  0
 34  2   44  2.7  7.0   7.9  22.6  0.81     "Never"    "Employed"  0  0  0
 44  1   36  5.8  5.7   6.6  29.3  0.88     "Never"    "Employed"  1  0  0
 47  4  239  7.8  5.0   5.1  22.8  0.84  …  "Current"  "Employed"  0  0  0
 54  2   58  6.2  7.7   7.2  23.3  0.81     "Never"    "Employed"  0  0  0
 63  2   44  8.0  6.0   6.8  26.6  0.88     "Never"    "Employed"  1  0  0
 

In [ ]:
function encode(column)
    labels=unique(column)
    onehot_matrix = zeros(Float64, length(column), length(labels))
    for (i, label) in enumerate(labels)
        onehot_matrix[column .== label, i] .= 1.0
    end
    return onehot_matrix
end

encode (generic function with 1 method)

In [ ]:
sample=train[1,:]
string_indices=filter(i->tryparse(Float64, string(sample[i])) === nothing,1:length(sample))
num_indices=filter(i->tryparse(Float64, string(sample[i])) !== nothing,1:length(sample))

num_data=Float64.(data[:,num_indices])
println(num_data[1,:])

string_data = Matrix{Float64}(undef, size(data, 1), 0)

for i in string_indices
    string_data = hcat(string_data, encode(data[:,i]))
end

encoded_data=hcat(num_data, string_data)


[31.0, 1.0, 45.0, 7.7, 6.8, 6.1, 33.4, 0.93, 112.0, 70.0, 62.0, 199.0, 58.0, 114.0, 102.0, 0.0, 0.0, 0.0]


1000000×42 Matrix{Float64}:
 31.0  1.0   45.0  7.7  6.8   6.1  33.4  …  1.0  0.0  0.0  1.0  0.0  0.0  0.0
 50.0  2.0   73.0  5.7  6.5   5.8  23.8     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 32.0  3.0  158.0  8.5  7.4   9.1  24.1     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 54.0  3.0   77.0  4.6  7.0   9.2  26.6     1.0  0.0  0.0  1.0  0.0  0.0  0.0
 54.0  1.0   55.0  5.7  6.2   5.1  28.8     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 42.0  1.0  100.0  4.4  6.4   5.3  25.5  …  0.0  1.0  0.0  0.0  1.0  0.0  0.0
 41.0  2.0  148.0  3.4  5.6   3.7  27.9     1.0  0.0  0.0  1.0  0.0  0.0  0.0
 51.0  3.0  102.0  4.0  7.3   5.5  27.1     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 34.0  2.0   44.0  2.7  7.0   7.9  22.6     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 44.0  1.0   36.0  5.8  5.7   6.6  29.3     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 47.0  4.0  239.0  7.8  5.0   5.1  22.8  …  1.0  0.0  0.0  1.0  0.0  0.0  0.0
 54.0  2.0   58.0  6.2  7.7   7.2  23.3     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 63.0  2.0   44.0  8.0  6.0   6.8  2

In [ ]:
use_train=encoded_data[1:size(train,1),:][end-21000:end,:]
use_valid=valid[end-21000:end,:]
use_test=encoded_data[size(train,1)+1:end,:]

300000×42 Matrix{Float64}:
 45.0  4.0  100.0  4.3  6.8   6.2  25.5  …  0.0  0.0  1.0  1.0  0.0  0.0  0.0
 35.0  1.0   87.0  3.5  4.6   9.0  28.6     0.0  1.0  0.0  0.0  0.0  0.0  1.0
 45.0  1.0   61.0  7.6  6.8   7.0  28.5     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 55.0  2.0   81.0  7.3  7.3   5.0  26.9     0.0  0.0  1.0  1.0  0.0  0.0  0.0
 77.0  2.0   29.0  7.3  7.6   8.5  22.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 32.0  1.0   14.0  4.8  7.0   9.1  25.3  …  0.0  0.0  1.0  1.0  0.0  0.0  0.0
 65.0  2.0   48.0  3.5  8.3   7.7  30.2     1.0  0.0  0.0  0.0  0.0  0.0  1.0
 39.0  2.0   46.0  4.8  8.4   3.0  27.7     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 52.0  1.0   88.0  6.6  7.5   6.8  25.9     0.0  1.0  0.0  1.0  0.0  0.0  0.0
 39.0  1.0   27.0  3.5  6.8   5.8  28.5     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 63.0  2.0   80.0  7.5  6.9   7.0  22.0  …  0.0  1.0  0.0  1.0  0.0  0.0  0.0
 49.0  2.0   51.0  5.6  6.4   7.1  26.0     0.0  1.0  0.0  0.0  1.0  0.0  0.0
 41.0  1.0  102.0  6.3  8.7   6.7  26

In [ ]:
W1=randn(128,size(use_train,2))*0.01
W2=randn(1,128)*0.01
b_1=randn(128,1)*0.01
b_2=randn(1,1)*0.01

1×1 Matrix{Float64}:
 0.001890549804625599

In [ ]:
for _ in 1:10
    for i in 1:size(use_train,1)
      z_1,x_1,x_2=forward(use_train[i:i,:]',W1,b_1,W2,b_2)
      loss=x_2 .- use_valid[i]
      W1,b_1,W2,b_2=backward(loss,use_train[i:i,:]',z_1,x_1,x_2,W1,b_1,W2,b_2,lr)
    end
end

In [ ]:
for i in 1:size(use_train,1)
  z_1,x_1,x_2=forward(use_train[i:i,:]',W1,b_1,W2,b_2)
  println(x_2[1]-use_valid[i])
end

Streaming output truncated to the last 5000 lines.
0.6004525918238625
-0.3778908543303915
0.5077095603332248
-0.34955690019384067
-0.39137783633806644
0.6210155656384878
-0.3211719441211548
-0.30504740519018236
0.42880915856081625
-0.37206028453785467
-0.5075870444275327
-0.3630142208923042
0.75601206265957
-0.35813156719386996
0.4598854308409479
-0.4852962393059247
0.4896358631878682
-0.5873191594888088
0.5041353427396257
0.5597731430383682
0.6036271325118283
-0.40808023252362646
-0.30024881774027345
0.5766433834522271
-0.4644648392709574
-0.34319120342158904
-0.3687885475592577
0.740041244717411
-0.4313983602067807
-0.3365331641598799
-0.3253171485984321
-0.38648507474950333
-0.45798528686774165
0.6728875132956555
-0.434734588705327
-0.630428320435969
-0.38575482787085513
-0.3901706397422132
-0.4075611329074925
-0.22778096098901768
-0.29838489549048397
0.6280180100582842
-0.41659446251033316
-0.2445567119504759
-0.2678928176278328
-0.40402169503489427
-0.4851780435987363
-0.251895783

In [ ]:
predictions = Float64[]
for i in 1:size(use_test,1)
  z_1,x_1,x_2=forward(use_test[i:i,:]',W1,b_1,W2,b_2)
  push!(predictions, x_2[1])
end

predictions_with_header = vcat(["dianosed_diabetes"], predictions)

raw_test_with_id = readdlm("/content/test.csv", ',')
id_column = raw_test_with_id[2:end, 1]
output_ids = string.(id_column)

final_id_column = vcat(["id"], output_ids)

output_data = hcat(final_id_column, predictions_with_header)

function to_csv(filename, data_matrix)
    open(filename, "w") do io
        DelimitedFiles.writedlm(io, data_matrix, ',')
    end
end
println(output_data[1,:],output_data[2,:],size(output_data,1))

to_csv("submission.csv", output_data)

Any["id", "dianosed_diabetes"]Any["700000", 0.651190133564275]300001
